5 Levels Of Text Splitting

In [1]:
#https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [1]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
#from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import RetrievalQA


from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\rachaudhari\OneDrive - Deloitte (O365D)\General\pythonenvs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import ollama
# llama3=ollama.pull('llama3')
# ollama.pull('mxbai-embed-large')

{'status': 'success'}

In [3]:
import os
pdf_files_to_process = []
for root, dirs, files in os.walk("../GenAI/data/twelfthphysics1/"):
    pdf_files_to_process.extend([os.path.join(root, file) for file in files if file.lower().endswith(".pdf")])

pdf_files_to_process

['../GenAI/data/twelfthphysics1/leph101.pdf',
 '../GenAI/data/twelfthphysics1/leph102.pdf',
 '../GenAI/data/twelfthphysics1/leph103.pdf',
 '../GenAI/data/twelfthphysics1/leph104.pdf',
 '../GenAI/data/twelfthphysics1/leph105.pdf',
 '../GenAI/data/twelfthphysics1/leph106.pdf',
 '../GenAI/data/twelfthphysics1/leph107.pdf',
 '../GenAI/data/twelfthphysics1/leph108.pdf',
 '../GenAI/data/twelfthphysics1/nr\\leph1an.pdf',
 '../GenAI/data/twelfthphysics1/nr\\leph1ps.pdf']

In [ ]:
#pdf_files_to_process

#remove unwanted files
pdf_files_to_process.pop(-1)
pdf_files_to_process.pop(-1)

'../GenAI/data/twelfthphysics1/nr\\leph1an.pdf'

In [6]:
pdf_files_to_process

['../GenAI/data/twelfthphysics1/leph101.pdf',
 '../GenAI/data/twelfthphysics1/leph102.pdf',
 '../GenAI/data/twelfthphysics1/leph103.pdf',
 '../GenAI/data/twelfthphysics1/leph104.pdf',
 '../GenAI/data/twelfthphysics1/leph105.pdf',
 '../GenAI/data/twelfthphysics1/leph106.pdf',
 '../GenAI/data/twelfthphysics1/leph107.pdf',
 '../GenAI/data/twelfthphysics1/leph108.pdf']

In [7]:
batch_docs = []

for files in pdf_files_to_process:
  pdfLoader = PyPDFLoader(files)
  documents = pdfLoader.load()
  batch_docs.extend(documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
docs = text_splitter.split_documents(batch_docs)

In [8]:
from langchain_ollama import OllamaEmbeddings

# llama3  # takes million years to when creating embeddings using this
# mxbai-embed-large
# nomic-embed-text
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)


####################################################################################################################
# FAISS.from_texts # for text data
# vectorstore = FAISS.from_documents(
#     docs, embedding=embeddings
# )

#save vector for later use
# vectorstore.save_local("../GenAI/data/langchain/vectorstores/faiss/twelfthphysics1_mxbai")
####################################################################################################################

#load vector from disk
# loading faiss index from local storage
vectorstore = FAISS.load_local("../GenAI/data/langchain/vectorstores/faiss/twelfthphysics1_mxbai", embeddings,allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()

# https://python.langchain.com/v0.1/docs/integrations/llms/ollama/
from langchain_community.llms import Ollama
llma = Ollama(model="llama3.1")
qa = RetrievalQA.from_chain_type(
    llm=llma, chain_type="stuff", retriever=retriever, return_source_documents=False)

In [ ]:
query = "What is DISPLACEMENT CURRENT?"
answer = qa({"query": query})

In [ ]:
# model: llama3.1
# embedding: mxbai-embed-large
# RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
print("Answer:",answer.get('result'))

### nomic-embed-text 

In [3]:
from langchain_ollama import OllamaEmbeddings

# llama3  # takes million years to when creating embeddings using this
# mxbai-embed-large
# nomic-embed-text
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

# load vector from disk

# loading faiss index from local storage
vectorstore = FAISS.load_local("../GenAI/data/langchain/vectorstores/faiss/twelfthphysics1", embeddings,allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()

# https://python.langchain.com/v0.1/docs/integrations/llms/ollama/
from langchain_community.llms import Ollama
llma = Ollama(model="llama3.1")
qa = RetrievalQA.from_chain_type(
    llm=llma, chain_type="stuff", retriever=retriever, return_source_documents=False)

C:\Users\rachaudhari\AppData\Local\Temp\ipykernel_8644\340576167.py:19: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llma = Ollama(model="llama3.1")


In [9]:
query = "What is DISPLACEMENT CURRENT?"
answer = qa({"query": query})

In [16]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: According to the text:

"the current, given by Eq. (8.4), is a new term, and is due to changing electric field (or electric displacement, an old term still used sometimes). It is, therefore, called displacement current or Maxwell’s displacement current."


In [25]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: According to the text:

"Maxwell’s displacement current. It is, therefore, called displacement current..."

Later it's also mentioned as "The displacement current has...".

Additionally, this is described in more detail later: 

"...the displacement current or Maxwell’s displacement current."

And finally:

"The displacement current has (literally) far reaching consequences."

So the answer from the textbook is:

DISPLACEMENT CURRENT


In [27]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: According to the provided context:

DISPLACEMENT CURRENT is also known as Maxwell's displacement current. It is the current due to changing electric field (or electric displacement) and is given by Eq. (8.4). In essence, it is a new term that represents the current carried by conductors due to the flow of charges.


In [29]:
# llama3.1
print("Answer:",answer.get('result'))

Answer: DISPLACEMENT CURRENT, or Maxwell's displacement current, is a type of electric current due to changing electric field (or electric displacement). It has the same physical effects as conduction current. The displacement current is given by Eq. (8.4) and plays an important role in Ampere-Maxwell law.


In [ ]:
# model: llama3.1
# embedding: nomic-embed-text
# RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
print("Answer:",answer.get('result'))

Answer: According to the context, Displacement Current is a term used to describe the current due to changing electric field or electric displacement. It's also referred to as Maxwell's displacement current. In other words, it's the current that arises from changes in the electric field over time, rather than being carried by conduction currents through conductors.


In [ ]:
# model: llama3.1
# embedding: mxbai-embed-large
# RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
print("Answer:",answer.get('result'))